In [1]:
# TODO which one?
#git clone https://github.com/lucidrains/iTransformer.git
#import iTransformer
import sys
sys.path.append('/vol/fob-vol7/nebenf21/reinbene/bene/MA/iTransformer') 
from iTransformer import iTransformer
import torch
from pathlib import Path

import torch
import torch.optim as optim
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter

from utils import data_handling, helpers, training_functions
import config
import pandas as pd

window_size = 96
pred_length = (96)

four_weeks = -24*7*4
print("Import succesfull")

#metrics_output_path = config.CONFIG_OUTPUT_PATH["itransformer"] / "itransformer_results_transfer_learning.csv"

metrics_output_path = config.CONFIG_OUTPUT_PATH["itransformer"] / "itransformer_results_transfer_learning_revin.csv"

Import succesfull


# Select dataset for transfer learning

In [2]:
# electricity dataset
data_dict = data_handling.load_electricity()

electricity_dict = {}
electricity_dict["dataloader_train"], electricity_dict["dataloader_validation"], electricity_dict["dataloader_test"] = data_handling.convert_data(data_dict, window_size, pred_length)

# create a smaller subset of the train dataset
electricity_dict["4_weeks_train"] = data_dict["train"][four_weeks:,:]
electricity_dict["4_weeks_train"] = data_handling.SlidingWindowTimeSeriesDataset(electricity_dict["4_weeks_train"] , window_size, pred_length)
electricity_dict["4_weeks_train"] = data_handling.DataLoader(electricity_dict["4_weeks_train"] , batch_size=32, shuffle=True)



# bavaria dataset
data_tensor = data_handling.load_bavaria_electricity()
data_dict, standadizer = data_handling.train_test_split_eu_elec(data_tensor, standardize=True)

# convert to datalaoder
bavaria_dict = {}
bavaria_dict["dataloader_train"], bavaria_dict["dataloader_validation"], bavaria_dict["dataloader_test"] = data_handling.convert_data(data_dict, window_size, pred_length)

# add fine-tuning datalaoders
# create a smaller subset of the train dataset
bavaria_dict["4_weeks_train"] = data_dict["train"][four_weeks:,:]
bavaria_dict["4_weeks_train"] = data_handling.SlidingWindowTimeSeriesDataset(bavaria_dict["4_weeks_train"] , window_size, pred_length)
bavaria_dict["4_weeks_train"] = data_handling.DataLoader(bavaria_dict["4_weeks_train"] , batch_size=32, shuffle=True)



# building genome project dataset
data_tensor = data_handling.load_genome_project_data()
data_dict, standadizer = data_handling.train_test_split_eu_elec(data_tensor, standardize=True)
# convert to datalaoder
gp_dict = {}
gp_dict["dataloader_train"], gp_dict["dataloader_validation"], gp_dict["dataloader_test"] = data_handling.convert_data(data_dict, window_size, pred_length)

# add fine-tuning datalaoders
# create a smaller subset of the train dataset
gp_dict["4_weeks_train"] = data_dict["train"][four_weeks:,:]
gp_dict["4_weeks_train"] = data_handling.SlidingWindowTimeSeriesDataset(gp_dict["4_weeks_train"] , window_size, pred_length)
gp_dict["4_weeks_train"] = data_handling.DataLoader(gp_dict["4_weeks_train"] , batch_size=32, shuffle=True)



# merge in dataset dict
datasets = {"ELD" : electricity_dict,
            "GP2" : gp_dict,
			"Bavaria" : bavaria_dict
            }


# define parameters for all models
best_parameters = {'depth': 2, 'dim': 256, 'dim_head': 56, 'heads': 4, 'attn_dropout': 0.2, 'ff_mult': 4, 'ff_dropout': 0.2, 
                        'num_mem_tokens': 4, 'learning_rate': 0.0005}

Feature batch shape: torch.Size([32, 96, 348])
Feature batch shape: torch.Size([32, 96, 59])
Feature batch shape: torch.Size([32, 96, 1454])


In [3]:
tl_setups = {
    "ELD_to_Bavaria" : [electricity_dict, bavaria_dict, "ELD", "Bavaria"], 
    "ELD_to_GP2" : [electricity_dict, gp_dict, "ELD", "GP2"],
    "Bavaria_to_ELD" : [bavaria_dict, electricity_dict, "Bavaria", "ELD"], 
    "Bavaria_to_GP2" : [bavaria_dict, gp_dict, "Bavaria", "GP2"], 
    "GP2_to_Bavaria": [gp_dict, bavaria_dict, "GP2", "Bavaria"], 
    "GP2_to_ELD" : [gp_dict, electricity_dict, "GP2", "ELD"]
     }


try:
    results_df = pd.read_csv(metrics_output_path, index_col=[0, 1, 2])
except FileNotFoundError:
    metrics = ["MSE", "MAE"]
    learning_scenarios = ["Zero-Shot", "four_weeks_tl", "full_tl", "full_training", "four_weeks_training"]
    index = pd.MultiIndex.from_product([tl_setups.keys(), learning_scenarios, metrics], names=["Setup", "Learning_scenario", "Metric"])
    results_df = pd.DataFrame(columns=["iTransformer"], index=index)

# Helper functions
def update_metrics(setup_name, model_name, learning_scenario, mae, mse):
    results_df.loc[(setup_name, learning_scenario, "MAE"), model_name] = mae
    results_df.loc[(setup_name, learning_scenario, "MSE"), model_name] = mse

# Zero-shot prediction on the test sets

In [4]:
for key, value in tl_setups.items():
    source_df = value[0]
    target_df = value[1]
    source_name = value[2]
    target_name = value[3]

    model_path = config.CONFIG_MODEL_LOCATION["itransformer"] / source_name / f"{source_name}_full_dataset_best_val_loss.pt"

    dataloader = target_df["dataloader_test"]

    inputs, _ = next(iter(dataloader))
    num_variates = inputs.size(2)

    model_config = {
            'num_variates': num_variates,
            'lookback_len': window_size,
            'depth': best_parameters["depth"],
            'dim': best_parameters["dim"],
            'num_tokens_per_variate': 1,
            'pred_length': pred_length,
            'dim_head': best_parameters["dim_head"],
            'heads': best_parameters["heads"],
            'attn_dropout': best_parameters["attn_dropout"],
            'ff_mult': best_parameters["ff_mult"],
            'ff_dropout': best_parameters["ff_dropout"],
            'num_mem_tokens': best_parameters["num_mem_tokens"],
            'use_reversible_instance_norm': True,
            'reversible_instance_norm_affine': True,
            'flash_attn': True
        }

    device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")


    checkpoint = torch.load(model_path,map_location='cpu')

    # values are all set to zero (beta) and one (gamma), array needs to be adapted to num_variates
    # learned affine parameters are series specific and need to be relearned for new series
    # value are kept at 1 and 0 for stationary normalization
    checkpoint["model_state_dict"]["reversible_instance_norm.beta"] = torch.zeros(num_variates, 1, dtype=torch.float)
    checkpoint["model_state_dict"]["reversible_instance_norm.gamma"] = torch.ones(num_variates, 1, dtype=torch.float)


    model = iTransformer(**model_config).to(device)
    model.load_state_dict(checkpoint['model_state_dict'])

    metrics = helpers.full_eval(model, dataloader, device)
    update_metrics(key, "iTransformer", "Zero-Shot", metrics[1], metrics[0])

results_df.to_csv(metrics_output_path)


Using device: cuda:1
Non-A100 GPU detected, using math or mem efficient attention if input tensor is on cuda


Epoch: Validating: 100%|██████████| 83/83 [00:00<00:00, 142.54it/s]
/tmp/ipykernel_14814/2943203897.py:21: PerformanceWarning: indexing past lexsort depth may impact performance.
  results_df.loc[(setup_name, learning_scenario, "MAE"), model_name] = mae
/tmp/ipykernel_14814/2943203897.py:22: PerformanceWarning: indexing past lexsort depth may impact performance.
  results_df.loc[(setup_name, learning_scenario, "MSE"), model_name] = mse


Using device: cuda:1


Epoch: Validating: 100%|██████████| 86/86 [00:03<00:00, 23.07it/s]


Using device: cuda:1


...

Epoch: Validating: 100%|██████████| 86/86 [00:03<00:00, 23.09it/s]


Using device: cuda:1


Epoch: Validating: 100%|██████████| 83/83 [00:00<00:00, 279.34it/s]


Using device: cuda:1


Epoch: Validating: 100%|██████████| 86/86 [00:00<00:00, 112.98it/s]


# Fine-tuning and predicitons on the test data

We fine tune for 5 epochs on different target datasets training sets length. After every epoch the training and validation loss is logged. For the final evaluation the model with the best validation loss is selected. 

In [5]:
def fine_tune(model, dataloader_train, dataloader_validation, device, epoch=1):

    # defining all needed instances
    optimizer = optim.Adam(model.parameters(), lr=0.0005)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
    writer = SummaryWriter(log_dir=config.CONFIG_LOGS_PATH["itransformer"])

    # run model training as mentioned in the original paper
    _, model = training_functions.train_one_epoch(epoch, model, device, dataloader_train, dataloader_validation, optimizer, \
                                                        scheduler, writer, save_model=False, validate=False)
    return model, device


epoch=10

for key, value in tl_setups.items():
    source_df = value[0]
    target_df = value[1]
    source_name = value[2]
    target_name = value[3]
    print(f"Using {source_name} as a source dataset.")

    model_path = config.CONFIG_MODEL_LOCATION["itransformer"] / source_name / f"{source_name}_full_dataset_best_val_loss.pt"


    # do test predictions for both target datasets
    dataloader_test = target_df["dataloader_test"]
    datalaoder_val = target_df["dataloader_validation"]
    fine_tune_dataloader = target_df["4_weeks_train"]  
    full_fine_tune_dataloader = target_df["dataloader_train"]        
      

    inputs, _ = next(iter(dataloader_test))
    num_variates = inputs.size(2)

    model_config = {
        'num_variates': num_variates,
        'lookback_len': window_size,
        'depth': best_parameters["depth"],
        'dim': best_parameters["dim"],
        'num_tokens_per_variate': 1,
        'pred_length': pred_length,
        'dim_head': best_parameters["dim_head"],
        'heads': best_parameters["heads"],
        'attn_dropout': best_parameters["attn_dropout"],
        'ff_mult': best_parameters["ff_mult"],
        'ff_dropout': best_parameters["ff_dropout"],
        'num_mem_tokens': best_parameters["num_mem_tokens"],
        'use_reversible_instance_norm': True,
        'reversible_instance_norm_affine': True,
        'flash_attn': True
    }

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")


    checkpoint = torch.load(model_path,map_location='cpu')

    # values are all set to zero (beta) and one (gamma), array needs to be adapted to num_variates
    # learned affine parameters are series specific and need to be relearned for new series
    # value are kept at 1 and 0 for stationary normalization
    checkpoint["model_state_dict"]["reversible_instance_norm.beta"] = torch.zeros(num_variates, 1, dtype=torch.float)
    checkpoint["model_state_dict"]["reversible_instance_norm.gamma"] = torch.ones(num_variates, 1, dtype=torch.float)


    model = iTransformer(**model_config).to(device)
    model.load_state_dict(checkpoint['model_state_dict'])

    
    # fne tuning on 4 weeks of target data
    model, device = fine_tune(model, fine_tune_dataloader, datalaoder_val, device, epoch=epoch)
    metrics_fine_tuned = helpers.full_eval(model, dataloader_test, device)
    update_metrics(key, "iTransformer", "four_weeks_tl", metrics_fine_tuned[1], metrics_fine_tuned[0])

    # train on full target dataset
    checkpoint = torch.load(model_path,map_location='cpu')

    # values are all set to zero (beta) and one (gamma), array needs to be adapted to num_variates
    # learned affine parameters are series specific and need to be relearned for new series
    # value are kept at 1 and 0 for stationary normalization
    checkpoint["model_state_dict"]["reversible_instance_norm.beta"] = torch.zeros(num_variates, 1, dtype=torch.float)
    checkpoint["model_state_dict"]["reversible_instance_norm.gamma"] = torch.ones(num_variates, 1, dtype=torch.float)

    model = iTransformer(**model_config).to(device)
    model.load_state_dict(checkpoint['model_state_dict'])

    # fne tuning on 4 weeks of target data
    model, device = fine_tune(model, full_fine_tune_dataloader, datalaoder_val, device, epoch=epoch)
    metrics_fine_tuned = helpers.full_eval(model, dataloader_test, device)
    update_metrics(key, "iTransformer", "full_tl", metrics_fine_tuned[1], metrics_fine_tuned[0])

results_df.to_csv(metrics_output_path)

Using ELD as a source dataset.
Using device: cuda:0


Epoch: Validating: 100%|██████████| 83/83 [00:00<00:00, 259.37it/s]
/tmp/ipykernel_14814/2943203897.py:21: PerformanceWarning: indexing past lexsort depth may impact performance.
  results_df.loc[(setup_name, learning_scenario, "MAE"), model_name] = mae
/tmp/ipykernel_14814/2943203897.py:22: PerformanceWarning: indexing past lexsort depth may impact performance.
  results_df.loc[(setup_name, learning_scenario, "MSE"), model_name] = mse
Epoch: Validating: 100%|██████████| 83/83 [00:00<00:00, 248.69it/s]


Using ELD as a source dataset.
Using device: cuda:0


Epoch: Validating: 100%|██████████| 86/86 [00:04<00:00, 20.06it/s]


Using Bavaria as a source dataset.
Using device: cuda:0


...

Epoch: Validating: 100%|██████████| 86/86 [00:04<00:00, 20.35it/s]


Using GP2 as a source dataset.
Using device: cuda:0


Epoch: Validating: 100%|██████████| 83/83 [00:00<00:00, 278.13it/s]


Using GP2 as a source dataset.
Using device: cuda:0


Epoch: Validating: 100%|██████████| 86/86 [00:00<00:00, 98.56it/s] 


In [6]:
results_df

iTransformer
Setup          Learning_scenario   Metric              
ELD_to_Bavaria Zero-Shot           MSE         0.241426
                                   MAE         0.461933
               four_weeks_tl       MSE         0.000939
                                   MAE         0.022352
               full_tl             MSE         0.000253
                                   MAE         0.008560
               full_training       MSE              NaN
                                   MAE              NaN
               four_weeks_training MSE              NaN
                                   MAE              NaN
ELD_to_GP2     Zero-Shot           MSE         0.684155
                                   MAE         0.482013
               four_weeks_tl       MSE         0.548209
                                   MAE         0.400982
               full_tl             MSE         0.465823
                                   MAE         0.357243
               full_training       MSE              NaN
                                   MAE              NaN
               four_weeks_training MSE              NaN
                                   MAE              NaN
Bavaria_to_ELD Zero-Shot           MSE         2.191598
                                   MAE         1.196772
               four_weeks_tl       MSE         0.274203
                                   MAE         0.359050
               full_tl             MSE         0.179394
                                   MAE         0.267813
               full_training       MSE              NaN
                                   MAE              NaN
               four_weeks_training MSE              NaN
                                   MAE              NaN
Bavaria_to_GP2 Zero-Shot           MSE         2.338648
                                   MAE         1.052081
               four_weeks_tl       MSE         0.552085
                                   MAE         0.414422
               full_tl             MSE         0.458592
                                   MAE         0.353943
               full_training       MSE              NaN
                                   MAE              NaN
               four_weeks_training MSE              NaN
                                   MAE              NaN
GP2_to_Bavaria Zero-Shot           MSE         0.226545
                                   MAE         0.435768
               four_weeks_tl       MSE         0.000472
                                   MAE         0.013904
               full_tl             MSE         0.000425
                                   MAE         0.013972
               full_training       MSE              NaN
                                   MAE              NaN
               four_weeks_training MSE              NaN
                                   MAE              NaN
GP2_to_ELD     Zero-Shot           MSE         0.331429
                                   MAE         0.392488
               four_weeks_tl       MSE         0.210335
                                   MAE         0.292538
               full_tl             MSE         0.179914
                                   MAE         0.262523
               full_training       MSE              NaN
                                   MAE              NaN
               four_weeks_training MSE              NaN
                                   MAE              NaN

In [7]:
#results_df